In [30]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

In [31]:
options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # 창 없이 실행
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
driver = webdriver.Chrome(options=options)

In [32]:
# Navigate to the website
url = "https://www.construction-europe.com/search?keyword=volvo"
driver.get(url)
time.sleep(2)

In [33]:
# Click the cookie button
driver.find_element(By.CSS_SELECTOR, '#cookieConsent > div > div > div > div.col-12.col-md-auto.text-center.py-2.align-self-center > button').click()

In [34]:
req = urllib.request.Request(url)
sourcecode = urllib.request.urlopen(url).read()
soup = BeautifulSoup(sourcecode, "html.parser")

In [35]:
# Click the options
brand = driver.find_element(By.ID, 'BrandFilter_3')
driver.execute_script("arguments[0].click();", brand)

type = driver.find_element(By.ID, 'TypeFilter_1')
driver.execute_script("arguments[0].click();", type)

category = driver.find_element(By.ID, 'CategoryFilter_1')
driver.execute_script("arguments[0].click();", category)

# Apply the selected options
driver.find_element(By.CSS_SELECTOR, '#filterOverlay > div > div > div > button.btn.btn-primary.btn-lg.btn-ApplyFilter').click()

In [36]:
# Scrape article details
full_address = []

for i in range(5):
    articles = soup.find_all("div", class_="khl-article-block-headline font-weight-bold h4")
    for article in articles:
        address = article.find("a")["href"]
        if address != 'articleurl' and address not in full_address:  # 'articleurl'이 아니고 중복되지 않은 경우에만 추가
            full_address.append(address)
        print(full_address)
        
    next_page_button = driver.find_element(By.CSS_SELECTOR, '#pagination > li.page-item.next > a').click()
    time.sleep(5)
    
    soup = BeautifulSoup(driver.page_source, "html.parser")

full_address

['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article']
['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article', 'https://www.construction-europe.com/news/heavy-wheeled-loaders-the-workhorse-of-the-construction-site/8028977.article']
['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article', 'https://www.construction-europe.com/news/heavy-wheeled-loaders-the-workhorse-of-the-construction-site/8028977.article', 'https://www.international-construction.com/news/why-electric-machines-could-be-a-game-changer-for-chinese-oems/8029203.article']
['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article', 'https://www.construction-europe.com/news/heavy-wheeled-loaders-the-workhorse-of-the-construction-site/8028977.article', 'https://www.international-construction.com/news/why-electric-m

['https://www.international-construction.com/news/world-s-top-ten-construction-oems-revealed/8029173.article',
 'https://www.construction-europe.com/news/heavy-wheeled-loaders-the-workhorse-of-the-construction-site/8028977.article',
 'https://www.international-construction.com/news/why-electric-machines-could-be-a-game-changer-for-chinese-oems/8029203.article',
 'https://www.construction-europe.com/news/holcim-orders-up-to-1-000-volvo-electric-trucks/8029110.article',
 'https://www.construction-europe.com/news/digging-with-a-twist-what-are-the-advantages-of-a-tiltrotator-/8027719.article',
 'https://www.international-construction.com/news/why-america-is-ready-for-electric-construction-equipment/8028903.article',
 'https://www.construction-europe.com/news/volvo-flies-in-electric-machine-for-alpine-project/8028555.article',
 'https://www.international-construction.com/news/online-exclusive-rokbak-investigates-electric-and-hydrogen-powered-articulated-trucks/8028398.article',
 'https://ww

In [38]:
article_data = []
for article_link in full_address:
    driver.get(article_link)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    release_date = ""
    title = ""
    content = ""
   
    # Scrape release date
    release_date_elem = soup.find("span", class_="PubDate")
    release_date = release_date_elem.text.strip() if release_date_elem else ""
            
    # Scrape news title
    title_elem = soup.find("h1", class_="khl-article-page-title")
    title = title_elem.text.strip() if title_elem else ""

    # Scrape article content
    first = soup.find("div", class_="col-12 khl-article-page-standfirst").text
    content_elem = soup.find("div", class_="col-12 khl-article-page-storybody")
    paragraphs = content_elem.find_all("p") if content_elem else []
    content = "\n".join([p.text.strip() for p in paragraphs])
    content_elem = soup.find("div", class_="col-12 khl-article-page-storybody")
    paragraphs = content_elem.find_all("p") if content_elem else []
    content = "\n".join([p.text.strip() for p in paragraphs])

    # Check if content is empty
    if not content:
        continue

    # Append article data to the list
    article_data.append({"Date": release_date, "Title": title, "Content": content})
driver.quit()

In [41]:
# Create DataFrame from article data
df = pd.DataFrame(article_data)
df

,Date,Title,Content
0,30 May 2023,World’s top ten construction OEMs revealed,"The latest Yellow Table, International Constru..."
1,26 May 2023,Heavy wheeled loaders: The workhorse of the co...,Heavy wheeled loaders have a home in almost ev...
2,25 May 2023,Why electric machines could be a game changer ...,"Forgive the generalization, but in the main th..."
3,22 May 2023,"Holcim orders up to 1,000 Volvo electric trucks",\n\nSwiss-based building materials company Hol...
4,18 May 2023,Digging with a twist: What are the advantages ...,"Popular in Scandinavia, and growing in other a..."
5,16 May 2023,Why America is ready for electric construction...,\nNorth America is ready for electric construc...
6,26 April 2023,Volvo flies in electric machine for Alpine pro...,The company delivered jaw-dropping images of t...
7,25 April 2023,Online exclusive: Rokbak investigates electric...,The idea of articulated trucks that can carry ...
8,20 April 2023,Volvo CE reports global sales growth of 11% in Q1,"The multinational, headquartered in Gothenburg..."
9,19 April 2023,Small but mighty: The role of mini and midi ex...,Many manufacturers in the construction industr...


In [42]:
# Save DataFrame to CSV
df.to_csv("volvo_article_final.csv", index=False)